In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import keras
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Dense, Dropout, Flatten
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from keras.applications.inception_v3 import InceptionV3

from keras.applications.xception import preprocess_input as xception_preprocessor
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception

Using TensorFlow backend.


In [2]:
import os
os.getcwd()

'D:\\ML\\Dog_Breed'

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
labels  = pd.read_csv('labels.csv')

In [5]:
labels['breed'].nunique()

120

In [6]:
labels

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


In [4]:
labels['breed'].value_counts()

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
great_pyrenees                    111
pomeranian                        111
basenji                           110
samoyed                           109
airedale                          107
tibetan_terrier                   107
leonberg                          106
cairn                             106
beagle                            105
japanese_spaniel                  105
australian_terrier                102
miniature_pinscher                102
blenheim_spaniel                  102
irish_wolfhound                   101
lakeland_terrier                   99
saluki                             99
papillon                           96
siberian_husky                     95
norwegian_elkhound                 95
whippet                            95
pug         

In [5]:
X_train = []
y_train = []
X_test = []
im_size = 299

In [6]:
for f, breed in tqdm(labels.values):
    img = cv2.imread('D:\\ML\\Dog_Breed\\train\\{}.jpg'.format(f))
    X_train.append(cv2.resize(img, (im_size, im_size)))

100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [01:30<00:00, 113.33it/s]


In [7]:
X_train = (np.array(X_train))

In [8]:
for i in tqdm(range(len(X_train))):
    X_train[i] = X_train[i] / 255

100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [00:12<00:00, 786.62it/s]


In [9]:
targets_series = pd.Series(labels['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)


In [10]:
one_hot_labels = np.asarray(one_hot) 

In [11]:
y_train = one_hot_labels

In [12]:
X = X_train
y = one_hot_labels


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=1)

In [16]:
# targets_series_train = pd.Series(y_train['breed'])
# targets_series_valid = pd.Series(y_valid['breed'])

# y_train = pd.get_dummies(targets_series_train, sparse = True)
# y_valid = pd.get_dummies(targets_series_valid, sparse = True)


In [17]:
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169

In [18]:
base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
predictions = Dense(120, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
from keras.optimizers import SGD 
optimizers = SGD(lr=0.001, momentum=0.9)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer= optimizers, 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 305, 305, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 150, 150, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

conv3_block4_1_conv (Conv2D)    (None, 37, 37, 128)  28672       conv3_block4_0_relu[0][0]        
__________________________________________________________________________________________________
conv3_block4_1_bn (BatchNormali (None, 37, 37, 128)  512         conv3_block4_1_conv[0][0]        
__________________________________________________________________________________________________
conv3_block4_1_relu (Activation (None, 37, 37, 128)  0           conv3_block4_1_bn[0][0]          
__________________________________________________________________________________________________
conv3_block4_2_conv (Conv2D)    (None, 37, 37, 32)   36864       conv3_block4_1_relu[0][0]        
__________________________________________________________________________________________________
conv3_block4_concat (Concatenat (None, 37, 37, 256)  0           conv3_block3_concat[0][0]        
                                                                 conv3_block4_2_conv[0][0]        
__________

conv4_block2_2_conv (Conv2D)    (None, 18, 18, 32)   36864       conv4_block2_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block2_concat (Concatenat (None, 18, 18, 320)  0           conv4_block1_concat[0][0]        
                                                                 conv4_block2_2_conv[0][0]        
__________________________________________________________________________________________________
conv4_block3_0_bn (BatchNormali (None, 18, 18, 320)  1280        conv4_block2_concat[0][0]        
__________________________________________________________________________________________________
conv4_block3_0_relu (Activation (None, 18, 18, 320)  0           conv4_block3_0_bn[0][0]          
__________________________________________________________________________________________________
conv4_block3_1_conv (Conv2D)    (None, 18, 18, 128)  40960       conv4_block3_0_relu[0][0]        
__________

__________________________________________________________________________________________________
conv4_block13_2_conv (Conv2D)   (None, 18, 18, 32)   36864       conv4_block13_1_relu[0][0]       
__________________________________________________________________________________________________
conv4_block13_concat (Concatena (None, 18, 18, 672)  0           conv4_block12_concat[0][0]       
                                                                 conv4_block13_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block14_0_bn (BatchNormal (None, 18, 18, 672)  2688        conv4_block13_concat[0][0]       
__________________________________________________________________________________________________
conv4_block14_0_relu (Activatio (None, 18, 18, 672)  0           conv4_block14_0_bn[0][0]         
__________________________________________________________________________________________________
conv4_bloc

conv4_block24_1_relu (Activatio (None, 18, 18, 128)  0           conv4_block24_1_bn[0][0]         
__________________________________________________________________________________________________
conv4_block24_2_conv (Conv2D)   (None, 18, 18, 32)   36864       conv4_block24_1_relu[0][0]       
__________________________________________________________________________________________________
conv4_block24_concat (Concatena (None, 18, 18, 1024) 0           conv4_block23_concat[0][0]       
                                                                 conv4_block24_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block25_0_bn (BatchNormal (None, 18, 18, 1024) 4096        conv4_block24_concat[0][0]       
__________________________________________________________________________________________________
conv4_block25_0_relu (Activatio (None, 18, 18, 1024) 0           conv4_block25_0_bn[0][0]         
__________

__________________________________________________________________________________________________
conv5_block3_0_bn (BatchNormali (None, 9, 9, 704)    2816        conv5_block2_concat[0][0]        
__________________________________________________________________________________________________
conv5_block3_0_relu (Activation (None, 9, 9, 704)    0           conv5_block3_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block3_1_conv (Conv2D)    (None, 9, 9, 128)    90112       conv5_block3_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block3_1_bn (BatchNormali (None, 9, 9, 128)    512         conv5_block3_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block3_1_relu (Activation (None, 9, 9, 128)    0           conv5_block3_1_bn[0][0]          
__________

                                                                 conv5_block13_2_conv[0][0]       
__________________________________________________________________________________________________
conv5_block14_0_bn (BatchNormal (None, 9, 9, 1056)   4224        conv5_block13_concat[0][0]       
__________________________________________________________________________________________________
conv5_block14_0_relu (Activatio (None, 9, 9, 1056)   0           conv5_block14_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block14_1_conv (Conv2D)   (None, 9, 9, 128)    135168      conv5_block14_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block14_1_bn (BatchNormal (None, 9, 9, 128)    512         conv5_block14_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_bloc

conv5_block24_concat (Concatena (None, 9, 9, 1408)   0           conv5_block23_concat[0][0]       
                                                                 conv5_block24_2_conv[0][0]       
__________________________________________________________________________________________________
conv5_block25_0_bn (BatchNormal (None, 9, 9, 1408)   5632        conv5_block24_concat[0][0]       
__________________________________________________________________________________________________
conv5_block25_0_relu (Activatio (None, 9, 9, 1408)   0           conv5_block25_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block25_1_conv (Conv2D)   (None, 9, 9, 128)    180224      conv5_block25_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block25_1_bn (BatchNormal (None, 9, 9, 128)    512         conv5_block25_1_conv[0][0]       
__________

In [19]:
model.fit(X, y, epochs = 100, verbose=1)

Epoch 1/100
 6496/10222 [==================>...........] - ETA: 2:33:50 - loss: 6.7807 - acc: 0.03 - ETA: 1:20:41 - loss: 6.7479 - acc: 0.03 - ETA: 56:12 - loss: 6.9284 - acc: 0.0208 - ETA: 43:59 - loss: 7.3390 - acc: 0.01 - ETA: 36:39 - loss: 7.4708 - acc: 0.01 - ETA: 31:45 - loss: 7.6152 - acc: 0.01 - ETA: 28:15 - loss: 7.9340 - acc: 0.00 - ETA: 25:36 - loss: 8.1095 - acc: 0.00 - ETA: 23:33 - loss: 8.1247 - acc: 0.00 - ETA: 21:54 - loss: 8.1901 - acc: 0.00 - ETA: 20:33 - loss: 8.2369 - acc: 0.01 - ETA: 19:24 - loss: 8.4412 - acc: 0.01 - ETA: 18:26 - loss: 8.5664 - acc: 0.01 - ETA: 17:36 - loss: 8.6895 - acc: 0.01 - ETA: 16:52 - loss: 8.7719 - acc: 0.02 - ETA: 16:14 - loss: 8.8958 - acc: 0.01 - ETA: 15:41 - loss: 9.0752 - acc: 0.02 - ETA: 15:10 - loss: 9.1837 - acc: 0.01 - ETA: 14:43 - loss: 9.3848 - acc: 0.01 - ETA: 14:10 - loss: 9.5317 - acc: 0.01 - ETA: 13:35 - loss: 9.6497 - acc: 0.01 - ETA: 13:04 - loss: 9.7502 - acc: 0.01 - ETA: 12:34 - loss: 9.9040 - acc: 0.01 - ETA: 12:08 - lo

 6528/10222 [==================>...........] - ETA: 3:05 - loss: 15.6164 - acc: 0.03 - ETA: 3:04 - loss: 15.8673 - acc: 0.01 - ETA: 3:04 - loss: 15.9509 - acc: 0.01 - ETA: 3:03 - loss: 15.9927 - acc: 0.00 - ETA: 3:03 - loss: 15.8163 - acc: 0.01 - ETA: 3:02 - loss: 15.7617 - acc: 0.02 - ETA: 3:02 - loss: 15.8126 - acc: 0.01 - ETA: 3:01 - loss: 15.8232 - acc: 0.01 - ETA: 3:01 - loss: 15.7555 - acc: 0.01 - ETA: 3:00 - loss: 15.7918 - acc: 0.01 - ETA: 3:00 - loss: 15.8215 - acc: 0.01 - ETA: 2:59 - loss: 15.7692 - acc: 0.01 - ETA: 2:58 - loss: 15.7573 - acc: 0.01 - ETA: 2:58 - loss: 15.7831 - acc: 0.01 - ETA: 2:57 - loss: 15.7797 - acc: 0.01 - ETA: 2:57 - loss: 15.6863 - acc: 0.01 - ETA: 2:56 - loss: 15.6821 - acc: 0.01 - ETA: 2:56 - loss: 15.7063 - acc: 0.01 - ETA: 2:55 - loss: 15.7280 - acc: 0.01 - ETA: 2:54 - loss: 15.7475 - acc: 0.01 - ETA: 2:54 - loss: 15.7325 - acc: 0.01 - ETA: 2:53 - loss: 15.7501 - acc: 0.01 - ETA: 2:53 - loss: 15.7661 - acc: 0.01 - ETA: 2:52 - loss: 15.7602 - acc: 

10222/10222 [==============================] - ETA: 1:06 - loss: 15.4427 - acc: 0.02 - ETA: 1:06 - loss: 15.4460 - acc: 0.02 - ETA: 1:05 - loss: 15.4463 - acc: 0.02 - ETA: 1:04 - loss: 15.4456 - acc: 0.02 - ETA: 1:04 - loss: 15.4393 - acc: 0.02 - ETA: 1:03 - loss: 15.4421 - acc: 0.02 - ETA: 1:03 - loss: 15.4428 - acc: 0.02 - ETA: 1:02 - loss: 15.4436 - acc: 0.02 - ETA: 1:01 - loss: 15.4429 - acc: 0.02 - ETA: 1:01 - loss: 15.4398 - acc: 0.02 - ETA: 1:00 - loss: 15.4388 - acc: 0.02 - ETA: 1:00 - loss: 15.4353 - acc: 0.02 - ETA: 59s - loss: 15.4331 - acc: 0.0243 - ETA: 59s - loss: 15.4328 - acc: 0.024 - ETA: 58s - loss: 15.4326 - acc: 0.024 - ETA: 57s - loss: 15.4352 - acc: 0.024 - ETA: 57s - loss: 15.4352 - acc: 0.024 - ETA: 56s - loss: 15.4340 - acc: 0.023 - ETA: 56s - loss: 15.4288 - acc: 0.024 - ETA: 55s - loss: 15.4278 - acc: 0.024 - ETA: 54s - loss: 15.4287 - acc: 0.024 - ETA: 54s - loss: 15.4299 - acc: 0.024 - ETA: 53s - loss: 15.4307 - acc: 0.024 - ETA: 53s - loss: 15.4306 - acc: 

 6528/10222 [==================>...........] - ETA: 3:03 - loss: 15.0855 - acc: 0.03 - ETA: 3:02 - loss: 14.7600 - acc: 0.04 - ETA: 3:02 - loss: 14.2637 - acc: 0.08 - ETA: 3:01 - loss: 14.4443 - acc: 0.07 - ETA: 3:01 - loss: 14.5067 - acc: 0.07 - ETA: 3:00 - loss: 14.4118 - acc: 0.07 - ETA: 3:00 - loss: 14.5332 - acc: 0.07 - ETA: 2:59 - loss: 14.4978 - acc: 0.07 - ETA: 2:59 - loss: 14.4776 - acc: 0.07 - ETA: 2:58 - loss: 14.4381 - acc: 0.08 - ETA: 2:58 - loss: 14.4995 - acc: 0.07 - ETA: 2:57 - loss: 14.5917 - acc: 0.07 - ETA: 2:57 - loss: 14.5727 - acc: 0.07 - ETA: 2:56 - loss: 14.5174 - acc: 0.07 - ETA: 2:55 - loss: 14.5383 - acc: 0.07 - ETA: 2:55 - loss: 14.5259 - acc: 0.07 - ETA: 2:54 - loss: 14.5307 - acc: 0.07 - ETA: 2:54 - loss: 14.6114 - acc: 0.06 - ETA: 2:53 - loss: 14.6118 - acc: 0.07 - ETA: 2:53 - loss: 14.5780 - acc: 0.07 - ETA: 2:52 - loss: 14.5744 - acc: 0.07 - ETA: 2:52 - loss: 14.5642 - acc: 0.07 - ETA: 2:51 - loss: 14.5704 - acc: 0.06 - ETA: 2:51 - loss: 14.5676 - acc: 

10222/10222 [==============================] - ETA: 1:06 - loss: 14.4801 - acc: 0.06 - ETA: 1:05 - loss: 14.4819 - acc: 0.06 - ETA: 1:05 - loss: 14.4896 - acc: 0.06 - ETA: 1:04 - loss: 14.4941 - acc: 0.06 - ETA: 1:03 - loss: 14.4977 - acc: 0.06 - ETA: 1:03 - loss: 14.4992 - acc: 0.06 - ETA: 1:02 - loss: 14.5064 - acc: 0.06 - ETA: 1:02 - loss: 14.5079 - acc: 0.06 - ETA: 1:01 - loss: 14.5121 - acc: 0.06 - ETA: 1:01 - loss: 14.5146 - acc: 0.06 - ETA: 1:00 - loss: 14.5195 - acc: 0.06 - ETA: 59s - loss: 14.5246 - acc: 0.0619 - ETA: 59s - loss: 14.5256 - acc: 0.061 - ETA: 58s - loss: 14.5322 - acc: 0.061 - ETA: 58s - loss: 14.5286 - acc: 0.061 - ETA: 57s - loss: 14.5259 - acc: 0.061 - ETA: 57s - loss: 14.5222 - acc: 0.061 - ETA: 56s - loss: 14.5250 - acc: 0.061 - ETA: 55s - loss: 14.5294 - acc: 0.061 - ETA: 55s - loss: 14.5292 - acc: 0.061 - ETA: 54s - loss: 14.5300 - acc: 0.061 - ETA: 54s - loss: 14.5270 - acc: 0.061 - ETA: 53s - loss: 14.5297 - acc: 0.061 - ETA: 52s - loss: 14.5316 - acc: 

...

10222/10222 [==============================] - ETA: 1:06 - loss: 8.4332 - acc: 0.461 - ETA: 1:05 - loss: 8.4392 - acc: 0.460 - ETA: 1:05 - loss: 8.4516 - acc: 0.460 - ETA: 1:04 - loss: 8.4484 - acc: 0.460 - ETA: 1:03 - loss: 8.4538 - acc: 0.459 - ETA: 1:03 - loss: 8.4640 - acc: 0.459 - ETA: 1:02 - loss: 8.4672 - acc: 0.459 - ETA: 1:02 - loss: 8.4796 - acc: 0.458 - ETA: 1:01 - loss: 8.4729 - acc: 0.458 - ETA: 1:00 - loss: 8.4804 - acc: 0.458 - ETA: 1:00 - loss: 8.4831 - acc: 0.458 - ETA: 59s - loss: 8.4788 - acc: 0.458 - ETA: 59s - loss: 8.4875 - acc: 0.45 - ETA: 58s - loss: 8.4971 - acc: 0.45 - ETA: 58s - loss: 8.4985 - acc: 0.45 - ETA: 57s - loss: 8.4965 - acc: 0.45 - ETA: 56s - loss: 8.5031 - acc: 0.45 - ETA: 56s - loss: 8.5079 - acc: 0.45 - ETA: 55s - loss: 8.5036 - acc: 0.45 - ETA: 55s - loss: 8.5019 - acc: 0.45 - ETA: 54s - loss: 8.4962 - acc: 0.45 - ETA: 54s - loss: 8.4965 - acc: 0.45 - ETA: 53s - loss: 8.4876 - acc: 0.45 - ETA: 52s - loss: 8.4924 - acc: 0.45 - ETA: 52s - loss: 8

 6528/10222 [==================>...........] - ETA: 3:03 - loss: 8.5627 - acc: 0.468 - ETA: 3:02 - loss: 9.8222 - acc: 0.390 - ETA: 3:02 - loss: 9.4024 - acc: 0.416 - ETA: 3:01 - loss: 9.4219 - acc: 0.406 - ETA: 3:00 - loss: 8.9534 - acc: 0.431 - ETA: 3:00 - loss: 8.7204 - acc: 0.447 - ETA: 3:00 - loss: 8.7286 - acc: 0.446 - ETA: 2:59 - loss: 8.6381 - acc: 0.453 - ETA: 2:58 - loss: 8.6857 - acc: 0.451 - ETA: 2:58 - loss: 8.6784 - acc: 0.446 - ETA: 2:57 - loss: 8.6221 - acc: 0.451 - ETA: 2:57 - loss: 8.6592 - acc: 0.450 - ETA: 2:56 - loss: 8.7719 - acc: 0.442 - ETA: 2:56 - loss: 8.7210 - acc: 0.446 - ETA: 2:55 - loss: 8.6756 - acc: 0.450 - ETA: 2:55 - loss: 8.7790 - acc: 0.443 - ETA: 2:54 - loss: 8.8552 - acc: 0.439 - ETA: 2:54 - loss: 8.8224 - acc: 0.441 - ETA: 2:53 - loss: 8.7557 - acc: 0.445 - ETA: 2:52 - loss: 8.8216 - acc: 0.442 - ETA: 2:52 - loss: 8.8093 - acc: 0.443 - ETA: 2:51 - loss: 8.6614 - acc: 0.453 - ETA: 2:51 - loss: 8.7009 - acc: 0.451 - ETA: 2:50 - loss: 8.7395 - acc: 0

10222/10222 [==============================] - ETA: 1:06 - loss: 8.3773 - acc: 0.465 - ETA: 1:05 - loss: 8.3806 - acc: 0.465 - ETA: 1:04 - loss: 8.3819 - acc: 0.465 - ETA: 1:04 - loss: 8.3828 - acc: 0.465 - ETA: 1:03 - loss: 8.3838 - acc: 0.465 - ETA: 1:03 - loss: 8.3846 - acc: 0.465 - ETA: 1:02 - loss: 8.3800 - acc: 0.465 - ETA: 1:02 - loss: 8.3805 - acc: 0.465 - ETA: 1:01 - loss: 8.3750 - acc: 0.466 - ETA: 1:00 - loss: 8.3823 - acc: 0.465 - ETA: 1:00 - loss: 8.3923 - acc: 0.465 - ETA: 59s - loss: 8.3997 - acc: 0.464 - ETA: 59s - loss: 8.4020 - acc: 0.46 - ETA: 58s - loss: 8.4113 - acc: 0.46 - ETA: 57s - loss: 8.3983 - acc: 0.46 - ETA: 57s - loss: 8.3907 - acc: 0.46 - ETA: 56s - loss: 8.3846 - acc: 0.46 - ETA: 56s - loss: 8.3854 - acc: 0.46 - ETA: 55s - loss: 8.3861 - acc: 0.46 - ETA: 55s - loss: 8.3869 - acc: 0.46 - ETA: 54s - loss: 8.3899 - acc: 0.46 - ETA: 53s - loss: 8.4039 - acc: 0.46 - ETA: 53s - loss: 8.4043 - acc: 0.46 - ETA: 52s - loss: 8.4037 - acc: 0.46 - ETA: 52s - loss: 8

 6528/10222 [==================>...........] - ETA: 3:07 - loss: 9.0998 - acc: 0.406 - ETA: 3:04 - loss: 8.3276 - acc: 0.468 - ETA: 3:03 - loss: 8.3834 - acc: 0.458 - ETA: 3:02 - loss: 8.0504 - acc: 0.484 - ETA: 3:01 - loss: 8.0754 - acc: 0.481 - ETA: 3:01 - loss: 8.3664 - acc: 0.463 - ETA: 3:00 - loss: 8.3226 - acc: 0.468 - ETA: 2:59 - loss: 8.3209 - acc: 0.468 - ETA: 2:59 - loss: 8.4599 - acc: 0.461 - ETA: 2:58 - loss: 8.5709 - acc: 0.456 - ETA: 2:57 - loss: 8.6159 - acc: 0.454 - ETA: 2:57 - loss: 8.5578 - acc: 0.458 - ETA: 2:56 - loss: 8.5969 - acc: 0.456 - ETA: 2:55 - loss: 8.7793 - acc: 0.444 - ETA: 2:55 - loss: 8.8321 - acc: 0.441 - ETA: 2:54 - loss: 8.8537 - acc: 0.439 - ETA: 2:54 - loss: 8.7774 - acc: 0.444 - ETA: 2:53 - loss: 8.8673 - acc: 0.439 - ETA: 2:53 - loss: 8.8033 - acc: 0.442 - ETA: 2:52 - loss: 8.7913 - acc: 0.443 - ETA: 2:52 - loss: 8.8051 - acc: 0.443 - ETA: 2:51 - loss: 8.7025 - acc: 0.450 - ETA: 2:50 - loss: 8.6624 - acc: 0.451 - ETA: 2:50 - loss: 8.5743 - acc: 0

10222/10222 [==============================] - ETA: 1:06 - loss: 8.4049 - acc: 0.463 - ETA: 1:05 - loss: 8.4148 - acc: 0.462 - ETA: 1:04 - loss: 8.4110 - acc: 0.462 - ETA: 1:04 - loss: 8.4093 - acc: 0.463 - ETA: 1:03 - loss: 8.4100 - acc: 0.463 - ETA: 1:03 - loss: 8.4108 - acc: 0.463 - ETA: 1:02 - loss: 8.4050 - acc: 0.463 - ETA: 1:02 - loss: 8.4081 - acc: 0.463 - ETA: 1:01 - loss: 8.4112 - acc: 0.463 - ETA: 1:00 - loss: 8.4119 - acc: 0.463 - ETA: 1:00 - loss: 8.4176 - acc: 0.462 - ETA: 59s - loss: 8.4232 - acc: 0.462 - ETA: 59s - loss: 8.4331 - acc: 0.46 - ETA: 58s - loss: 8.4241 - acc: 0.46 - ETA: 57s - loss: 8.4178 - acc: 0.46 - ETA: 57s - loss: 8.4276 - acc: 0.46 - ETA: 56s - loss: 8.4292 - acc: 0.46 - ETA: 56s - loss: 8.4232 - acc: 0.46 - ETA: 55s - loss: 8.4198 - acc: 0.46 - ETA: 55s - loss: 8.4160 - acc: 0.46 - ETA: 54s - loss: 8.4205 - acc: 0.46 - ETA: 53s - loss: 8.4153 - acc: 0.46 - ETA: 53s - loss: 8.4115 - acc: 0.46 - ETA: 52s - loss: 8.4190 - acc: 0.46 - ETA: 52s - loss: 8

In [20]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 zero_padding2d_1
2 conv1/conv
3 conv1/bn
4 conv1/relu
5 zero_padding2d_2
6 pool1
7 conv2_block1_0_bn
8 conv2_block1_0_relu
9 conv2_block1_1_conv
10 conv2_block1_1_bn
11 conv2_block1_1_relu
12 conv2_block1_2_conv
13 conv2_block1_concat
14 conv2_block2_0_bn
15 conv2_block2_0_relu
16 conv2_block2_1_conv
17 conv2_block2_1_bn
18 conv2_block2_1_relu
19 conv2_block2_2_conv
20 conv2_block2_concat
21 conv2_block3_0_bn
22 conv2_block3_0_relu
23 conv2_block3_1_conv
24 conv2_block3_1_bn
25 conv2_block3_1_relu
26 conv2_block3_2_conv
27 conv2_block3_concat
28 conv2_block4_0_bn
29 conv2_block4_0_relu
30 conv2_block4_1_conv
31 conv2_block4_1_bn
32 conv2_block4_1_relu
33 conv2_block4_2_conv
34 conv2_block4_concat
35 conv2_block5_0_bn
36 conv2_block5_0_relu
37 conv2_block5_1_conv
38 conv2_block5_1_bn
39 conv2_block5_1_relu
40 conv2_block5_2_conv
41 conv2_block5_concat
42 conv2_block6_0_bn
43 conv2_block6_0_relu
44 conv2_block6_1_conv
45 conv2_block6_1_bn
46 conv2_block6_1_relu
47 conv2_block

In [21]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [22]:
callbacks_list = [keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1)]

In [24]:
model.fit(X, y, epochs = 10, verbose=1)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[32,18,18,1248] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_1/SGD/gradients/zeros_271-0-1-TransposeNCHWToNHWC-LayoutOptimizer = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/SGD/gradients/zeros_271, PermConstNCHWToNHWC-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_1/mul/_7327 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_27293_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_

In [48]:
 test_labels = pd.read_csv('sample_submission.csv')

In [50]:
X_test = []
for f in tqdm(test_labels['id'].values):
    img = cv2.imread('D:\\ML\\Dog_Breed\\test\\{}.jpg'.format(f))
    #label = one_hot_labels[i]
    X_test.append(cv2.resize(img, (im_size, im_size)))
    #y_train.append(label)

100%|███████████████████████████████████████████████████████████████████████████| 10357/10357 [01:42<00:00, 100.76it/s]


In [51]:
X_test  = np.array(X_test)

In [52]:
for i in tqdm(range(len(X_test))):
    X_test[i] = X_test[i] / 255

100%|███████████████████████████████████████████████████████████████████████████| 10357/10357 [00:14<00:00, 726.23it/s]


In [33]:
#test_labels = pd.read_csv('sample_submission.csv')

In [34]:
preds  = model.predict(X_test, verbose = 1)

10357/10357 [==============================] - ETA: 14:49: - ETA: 7:31:15 - ETA: 5:03:4 - ETA: 3:49:5 - ETA: 3:05:4 - ETA: 2:36:1 - ETA: 2:15:0 - ETA: 1:59:1 - ETA: 1:46:5 - ETA: 1:37:0 - ETA: 1:28:5 - ETA: 1:22:1 - ETA: 1:16:2 - ETA: 1:11:3 - ETA: 1:07:1 - ETA: 1:03:3 - ETA: 1:00:1 - ETA: 57:23  - ETA: 54:4 - ETA: 52:2 - ETA: 50:1 - ETA: 48:1 - ETA: 46:2 - ETA: 44:5 - ETA: 43:1 - ETA: 41:5 - ETA: 40:3 - ETA: 39:2 - ETA: 38:1 - ETA: 37:1 - ETA: 36:1 - ETA: 35:1 - ETA: 34:2 - ETA: 33:3 - ETA: 32:4 - ETA: 32:0 - ETA: 31:2 - ETA: 30:4 - ETA: 30:0 - ETA: 29:2 - ETA: 28:5 - ETA: 28:2 - ETA: 27:4 - ETA: 27:1 - ETA: 26:4 - ETA: 26:2 - ETA: 25:5 - ETA: 25:2 - ETA: 25:0 - ETA: 24:4 - ETA: 24:1 - ETA: 23:5 - ETA: 23:3 - ETA: 23:1 - ETA: 22:5 - ETA: 22:3 - ETA: 22:1 - ETA: 21:5 - ETA: 21:3 - ETA: 21:2 - ETA: 21:0 - ETA: 20:4 - ETA: 20:3 - ETA: 20:1 - ETA: 20:0 - ETA: 19:4 - ETA: 19:3 - ETA: 19:2 - ETA: 19:0 - ETA: 18:5 - ETA: 18:4 - ETA: 18:2 - ETA: 18:1 - ETA: 18:0 - ETA: 17:5 - ETA: 17:4 - ETA:

In [1]:
submission = pd.DataFrame(preds)

NameError: name 'pd' is not defined

In [36]:
submission.columns = one_hot.columns

In [37]:
submission.insert(0, 'id', test_labels['id'])

In [41]:
submission.to_csv('submission_resnet169_once.csv', index= False)

In [43]:
submission.shape

(10357, 121)

In [15]:
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

batch_size = 5
epochs = 30
num_classes = 120
SEED = 1234

def generate_features(model_info, data, labels, datagen):
    print("generating features...")
    datagen.preprocessing_function = model_info["preprocessor"]
    generator = datagen.flow(data, labels, shuffle=False, batch_size=batch_size, seed=model_info["seed"])
    bottleneck_model = model_info["model"](weights='imagenet', include_top=False, input_shape=model_info["input_shape"], pooling=model_info["pooling"])
    return bottleneck_model.predict_generator(generator)

In [28]:
models = {
    "InceptionV3": {
        "model": InceptionV3,
        "preprocessor": inception_v3_preprocessor,
        "input_shape": (299,299,3),
        "seed": 1234,
        "pooling": "avg"
    },
    "Xception": {
        "model": Xception, 
        "preprocessor": xception_preprocessor,
        "input_shape": (299,299,3),
        "seed": 5512,
        "pooling": "avg"
    }
}



In [29]:
import os
import time
for model_name, model in models.items():
    print("Predicting : {}".format(model_name))
    filename = model_name + '_features.npy'
    validfilename = model_name + '_validfeatures.npy'
    if os.path.exists(filename):
        features = np.load(filename)
        validation_features = np.load(validfilename)
    else:
        train_datagen = ImageDataGenerator(
                zoom_range = 0.3,
                width_shift_range=0.1,
                height_shift_range=0.1)
        validation_datagen = ImageDataGenerator()
        features = generate_features( model, X_train, y_train, train_datagen)
        validation_features = generate_features(model, X_valid, y_valid, validation_datagen)
        #test_features = generate_features(model, X_valid, y_valid, validation_datagen)
        np.save(filename, features)
        np.save(validfilename, validation_features)
    
    # Now that we have created or loaded the features  we need to do some predictions.
    start_time = time.time()
    
    logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
    logreg.fit(features, (y_train * range(num_classes)).sum(axis=1))

    model["predict_proba"] = logreg.predict_proba(validation_features)
    end_time = time.time()
    print('Training time : {} {}'.format(np.round((end_time-start_time)/60, 2),' minutes'))

Predicting : InceptionV3


c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Training time : 0.29  minutes
Predicting : Xception
generating features...
32473088/83683744 [==========>...................] - ETA:  - ETA: 24:4 - ETA: 23:5 - ETA: 27:4 - ETA: 27:3 - ETA: 25:0 - ETA: 27:3 - ETA: 27:1 - ETA: 23:1 - ETA: 14:3 - ETA: 15:1 - ETA: 10:5 - ETA: 10:0 - ETA: 9:4 - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: -

c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Training time : 0.28  minutes


In [31]:
probas = [ model["predict_proba"] for model_name, model in models.items() ]

avgprobas = np.average(probas, axis=0, weights=[1,1])

print('ensemble validation logLoss : {}'.format(log_loss(y_valid, avgprobas)))

ensemble validation logLoss : 4.751415420335388


In [33]:
features  = np.hstack( [ np.load(model_name + '_features.npy') for model_name, model in models.items() ])
validation = np.hstack( [ np.load(model_name + '_validfeatures.npy') for model_name, model in models.items() ])

logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(features, (y_train * range(num_classes)).sum(axis=1))

predict_probs = logreg.predict_proba(validation)

print('ensemble of features va logLoss : {}'.format(log_loss(y_valid, predict_probs)))

c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


ensemble of features va logLoss : 4.809159892335044


In [38]:
X_train.shape

(8177, 299, 299, 3)

In [40]:
X_valid.shape

(2045, 299, 299, 3)

In [41]:
X_test = np.array(X_test) /255

In [42]:
X_test.shape

(0,)

# InceptionV3

In [14]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.densenet import DenseNet169
from keras.applications.vgg16 import VGG16


In [15]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

In [16]:
# x = base_model.output
# x = Flatten()(x)
# predictions = Dense(120, activation='softmax')(x)

# # This is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)

# # First: train only the top layers (which were randomly initialized)
# for layer in base_model.layers:
#     layer.trainable = False

# model.compile(loss='categorical_crossentropy', 
#               optimizer='adam', 
#               metrics=['accuracy'])

# callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
# model.summary()

In [17]:
x = base_model.output
x = Flatten()(x)
#x = Dropout(0.3)(x)
#x = Dense(1204, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(120, activation="softmax")(x)
from keras.optimizers import SGD 
optimizers = SGD(lr=0.001, momentum=0.9)

In [18]:
model = Model(inputs=base_model.input, outputs=predictions)

In [19]:
for layer in base_model.layers:
    layer.trainable = False

In [20]:
model.compile(optimizer= optimizers, loss='categorical_crossentropy')

In [21]:
for layer in base_model.layers:
    layer.trainable = False

In [22]:
callbacks_list = [keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1)]

In [23]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 35, 35, 64)   192         conv2d_20[0][0]                  
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 35, 35, 64)   192         conv2d_22[0][0]                  
__________________________________________________________________________________________________
batch_normalization_25 (BatchNo (None, 35, 35, 96)   288         conv2d_25[0][0]                  
__________________________________________________________________________________________________
batch_normalization_26 (BatchNo (None, 35, 35, 64)   192         conv2d_26[0][0]                  
__________________________________________________________________________________________________
activation_20 (Activation)      (None, 35, 35, 64)   0           batch_normalization_20[0][0]     
__________

__________________________________________________________________________________________________
batch_normalization_41 (BatchNo (None, 17, 17, 192)  576         conv2d_41[0][0]                  
__________________________________________________________________________________________________
batch_normalization_44 (BatchNo (None, 17, 17, 192)  576         conv2d_44[0][0]                  
__________________________________________________________________________________________________
batch_normalization_49 (BatchNo (None, 17, 17, 192)  576         conv2d_49[0][0]                  
__________________________________________________________________________________________________
batch_normalization_50 (BatchNo (None, 17, 17, 192)  576         conv2d_50[0][0]                  
__________________________________________________________________________________________________
activation_41 (Activation)      (None, 17, 17, 192)  0           batch_normalization_41[0][0]     
__________

batch_normalization_74 (BatchNo (None, 17, 17, 192)  576         conv2d_74[0][0]                  
__________________________________________________________________________________________________
activation_74 (Activation)      (None, 17, 17, 192)  0           batch_normalization_74[0][0]     
__________________________________________________________________________________________________
conv2d_71 (Conv2D)              (None, 17, 17, 192)  147456      mixed7[0][0]                     
__________________________________________________________________________________________________
conv2d_75 (Conv2D)              (None, 17, 17, 192)  258048      activation_74[0][0]              
__________________________________________________________________________________________________
batch_normalization_71 (BatchNo (None, 17, 17, 192)  576         conv2d_71[0][0]                  
__________________________________________________________________________________________________
batch_norm

                                                                 mixed9_1[0][0]                   
                                                                 concatenate_2[0][0]              
                                                                 activation_94[0][0]              
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 131072)       0           mixed10[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 131072)       0           flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 120)          15728760    dropout_1[0][0]                  
Total params: 37,531,544
Trainable params: 15,728,760
Non-trainable params: 21,802,784
______________________

In [25]:
X_train.shape

(8688, 299, 299, 3)

In [26]:
# y_train = np.asarray(y_train) 
# y_valid = np.asarray(y_valid) 

In [27]:
X_valid.shape

(1534, 299, 299, 3)

In [28]:
model.fit(X_train, y_train, batch_size= 10,epochs=30, verbose=1, validation_data = (X_valid, y_valid) ,callbacks= callbacks_list)

Train on 8688 samples, validate on 1534 samples
Epoch 1/30


2010/8688 [=====>........................] - ETA: 4:01:31 - loss: 4.8028 - acc: 0.10 - ETA: 2:01:56 - loss: 9.7456 - acc: 0.05 - ETA: 1:22:08 - loss: 11.6602 - acc: 0.033 - ETA: 1:02:05 - loss: 12.0534 - acc: 0.025 - ETA: 50:05 - loss: 12.6399 - acc: 0.0200  - ETA: 42:05 - loss: 13.1893 - acc: 0.016 - ETA: 36:22 - loss: 13.6077 - acc: 0.014 - ETA: 32:03 - loss: 13.9215 - acc: 0.012 - ETA: 28:44 - loss: 14.1007 - acc: 0.011 - ETA: 26:04 - loss: 14.3024 - acc: 0.010 - ETA: 23:53 - loss: 14.3210 - acc: 0.018 - ETA: 22:04 - loss: 14.4707 - acc: 0.016 - ETA: 20:31 - loss: 14.4172 - acc: 0.023 - ETA: 19:11 - loss: 14.3864 - acc: 0.028 - ETA: 18:03 - loss: 14.5018 - acc: 0.026 - ETA: 17:03 - loss: 14.6028 - acc: 0.025 - ETA: 16:10 - loss: 14.6223 - acc: 0.023 - ETA: 15:22 - loss: 14.7054 - acc: 0.022 - ETA: 14:40 - loss: 14.7798 - acc: 0.021 - ETA: 14:02 - loss: 14.7934 - acc: 0.020 - ETA: 13:27 - loss: 14.8057 - acc: 0.019 - ETA: 12:56 - loss: 14.8653 - acc: 0.018 - ETA: 12:28 - loss: 14.919

8130/8688 [===========================>..] - ETA: 49s - loss: 15.9180 - acc: 0.009 - ETA: 49s - loss: 15.9183 - acc: 0.009 - ETA: 48s - loss: 15.9186 - acc: 0.009 - ETA: 48s - loss: 15.9189 - acc: 0.009 - ETA: 48s - loss: 15.9193 - acc: 0.009 - ETA: 48s - loss: 15.9196 - acc: 0.009 - ETA: 48s - loss: 15.9199 - acc: 0.009 - ETA: 47s - loss: 15.9202 - acc: 0.009 - ETA: 47s - loss: 15.9179 - acc: 0.009 - ETA: 47s - loss: 15.9183 - acc: 0.009 - ETA: 47s - loss: 15.9186 - acc: 0.009 - ETA: 46s - loss: 15.9189 - acc: 0.009 - ETA: 46s - loss: 15.9192 - acc: 0.009 - ETA: 46s - loss: 15.9195 - acc: 0.009 - ETA: 46s - loss: 15.9199 - acc: 0.009 - ETA: 46s - loss: 15.9202 - acc: 0.009 - ETA: 46s - loss: 15.9205 - acc: 0.009 - ETA: 45s - loss: 15.9208 - acc: 0.009 - ETA: 45s - loss: 15.9211 - acc: 0.009 - ETA: 45s - loss: 15.9214 - acc: 0.009 - ETA: 45s - loss: 15.9192 - acc: 0.009 - ETA: 45s - loss: 15.9195 - acc: 0.009 - ETA: 44s - loss: 15.9198 - acc: 0.009 - ETA: 44s - loss: 15.9201 - acc: 0.0

8688/8688 [==============================] - ETA: 10s - loss: 15.9230 - acc: 0.010 - ETA: 9s - loss: 15.9233 - acc: 0.010 - ETA: 9s - loss: 15.9235 - acc: 0.01 - ETA: 9s - loss: 15.9237 - acc: 0.01 - ETA: 9s - loss: 15.9240 - acc: 0.01 - ETA: 9s - loss: 15.9242 - acc: 0.01 - ETA: 8s - loss: 15.9245 - acc: 0.01 - ETA: 8s - loss: 15.9227 - acc: 0.01 - ETA: 8s - loss: 15.9230 - acc: 0.01 - ETA: 8s - loss: 15.9232 - acc: 0.01 - ETA: 8s - loss: 15.9234 - acc: 0.01 - ETA: 8s - loss: 15.9237 - acc: 0.01 - ETA: 7s - loss: 15.9239 - acc: 0.01 - ETA: 7s - loss: 15.9241 - acc: 0.01 - ETA: 7s - loss: 15.9244 - acc: 0.01 - ETA: 7s - loss: 15.9246 - acc: 0.01 - ETA: 7s - loss: 15.9248 - acc: 0.01 - ETA: 6s - loss: 15.9251 - acc: 0.01 - ETA: 6s - loss: 15.9234 - acc: 0.01 - ETA: 6s - loss: 15.9236 - acc: 0.01 - ETA: 6s - loss: 15.9238 - acc: 0.01 - ETA: 6s - loss: 15.9241 - acc: 0.01 - ETA: 6s - loss: 15.9243 - acc: 0.01 - ETA: 5s - loss: 15.9245 - acc: 0.01 - ETA: 5s - loss: 15.9248 - acc: 0.01 - ET

2040/8688 [======>.......................] - ETA: 2:15 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:22 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:19 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:21 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:20 - loss: 15.7957 - acc: 0.0200   - ETA: 2:21 - loss: 15.8495 - acc: 0.01 - ETA: 2:20 - loss: 15.8878 - acc: 0.01 - ETA: 2:20 - loss: 15.9166 - acc: 0.01 - ETA: 2:20 - loss: 15.9390 - acc: 0.01 - ETA: 2:20 - loss: 15.9569 - acc: 0.01 - ETA: 2:20 - loss: 15.8250 - acc: 0.01 - ETA: 2:20 - loss: 15.8495 - acc: 0.01 - ETA: 2:19 - loss: 15.8701 - acc: 0.01 - ETA: 2:19 - loss: 15.8878 - acc: 0.01 - ETA: 2:19 - loss: 15.9032 - acc: 0.01 - ETA: 2:18 - loss: 15.9166 - acc: 0.01 - ETA: 2:18 - loss: 15.9285 - acc: 0.01 - ETA: 2:18 - loss: 15.9390 - acc: 0.01 - ETA: 2:18 - loss: 15.9484 - acc: 0.01 - ETA: 2:18 - loss: 15.9569 - acc: 0.01 - ETA: 2:18 - loss: 15.9646 - acc: 0.00 - ETA: 2:18 - loss: 15.9716 - acc: 0.00 - ETA: 2:17 - loss: 15.9779 - acc: 0.00 - ETA: 2:17 - los

...

8150/8688 [===========================>..] - ETA: 42s - loss: 15.9607 - acc: 0.009 - ETA: 41s - loss: 15.9610 - acc: 0.009 - ETA: 41s - loss: 15.9612 - acc: 0.009 - ETA: 41s - loss: 15.9615 - acc: 0.009 - ETA: 41s - loss: 15.9617 - acc: 0.009 - ETA: 41s - loss: 15.9620 - acc: 0.009 - ETA: 41s - loss: 15.9622 - acc: 0.009 - ETA: 40s - loss: 15.9599 - acc: 0.009 - ETA: 40s - loss: 15.9601 - acc: 0.009 - ETA: 40s - loss: 15.9604 - acc: 0.009 - ETA: 40s - loss: 15.9606 - acc: 0.009 - ETA: 40s - loss: 15.9609 - acc: 0.009 - ETA: 40s - loss: 15.9611 - acc: 0.009 - ETA: 39s - loss: 15.9614 - acc: 0.009 - ETA: 39s - loss: 15.9616 - acc: 0.009 - ETA: 39s - loss: 15.9619 - acc: 0.009 - ETA: 39s - loss: 15.9621 - acc: 0.009 - ETA: 39s - loss: 15.9598 - acc: 0.009 - ETA: 39s - loss: 15.9601 - acc: 0.009 - ETA: 38s - loss: 15.9603 - acc: 0.009 - ETA: 38s - loss: 15.9606 - acc: 0.009 - ETA: 38s - loss: 15.9608 - acc: 0.009 - ETA: 38s - loss: 15.9611 - acc: 0.009 - ETA: 38s - loss: 15.9613 - acc: 0.0

8688/8688 [==============================] - ETA: 8s - loss: 15.9494 - acc: 0.01 - ETA: 8s - loss: 15.9496 - acc: 0.01 - ETA: 8s - loss: 15.9498 - acc: 0.01 - ETA: 8s - loss: 15.9500 - acc: 0.01 - ETA: 7s - loss: 15.9502 - acc: 0.01 - ETA: 7s - loss: 15.9504 - acc: 0.01 - ETA: 7s - loss: 15.9506 - acc: 0.01 - ETA: 7s - loss: 15.9508 - acc: 0.01 - ETA: 7s - loss: 15.9510 - acc: 0.01 - ETA: 7s - loss: 15.9512 - acc: 0.01 - ETA: 6s - loss: 15.9494 - acc: 0.01 - ETA: 6s - loss: 15.9496 - acc: 0.01 - ETA: 6s - loss: 15.9498 - acc: 0.01 - ETA: 6s - loss: 15.9501 - acc: 0.01 - ETA: 6s - loss: 15.9503 - acc: 0.01 - ETA: 6s - loss: 15.9485 - acc: 0.01 - ETA: 6s - loss: 15.9487 - acc: 0.01 - ETA: 5s - loss: 15.9489 - acc: 0.01 - ETA: 5s - loss: 15.9491 - acc: 0.01 - ETA: 5s - loss: 15.9493 - acc: 0.01 - ETA: 5s - loss: 15.9495 - acc: 0.01 - ETA: 5s - loss: 15.9497 - acc: 0.01 - ETA: 5s - loss: 15.9499 - acc: 0.01 - ETA: 4s - loss: 15.9501 - acc: 0.01 - ETA: 4s - loss: 15.9503 - acc: 0.01 - ETA: 

2040/8688 [======>.......................] - ETA: 2:21 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:22 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:23 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:20 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:22 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:22 - loss: 16.1181 - acc: 0.0000e+ - ETA: 2:21 - loss: 15.8878 - acc: 0.0143   - ETA: 2:20 - loss: 15.9166 - acc: 0.01 - ETA: 2:19 - loss: 15.9390 - acc: 0.01 - ETA: 2:20 - loss: 15.9569 - acc: 0.01 - ETA: 2:20 - loss: 15.9716 - acc: 0.00 - ETA: 2:20 - loss: 15.9838 - acc: 0.00 - ETA: 2:19 - loss: 15.9941 - acc: 0.00 - ETA: 2:18 - loss: 16.0030 - acc: 0.00 - ETA: 2:19 - loss: 16.0106 - acc: 0.00 - ETA: 2:18 - loss: 16.0174 - acc: 0.00 - ETA: 2:18 - loss: 16.0233 - acc: 0.00 - ETA: 2:18 - loss: 16.0286 - acc: 0.00 - ETA: 2:18 - loss: 16.0333 - acc: 0.00 - ETA: 2:18 - loss: 16.0375 - acc: 0.00 - ETA: 2:17 - loss: 16.0413 - acc: 0.00 - ETA: 2:18 - loss: 16.0448 - acc: 0.00 - ETA: 2:17 - loss: 15.9779 - acc: 0.00 - ETA: 2:

8160/8688 [===========================>..] - ETA: 41s - loss: 15.9366 - acc: 0.011 - ETA: 41s - loss: 15.9369 - acc: 0.011 - ETA: 41s - loss: 15.9346 - acc: 0.011 - ETA: 41s - loss: 15.9349 - acc: 0.011 - ETA: 41s - loss: 15.9352 - acc: 0.011 - ETA: 41s - loss: 15.9355 - acc: 0.011 - ETA: 40s - loss: 15.9358 - acc: 0.011 - ETA: 41s - loss: 15.9361 - acc: 0.011 - ETA: 41s - loss: 15.9364 - acc: 0.011 - ETA: 41s - loss: 15.9367 - acc: 0.011 - ETA: 41s - loss: 15.9370 - acc: 0.011 - ETA: 41s - loss: 15.9373 - acc: 0.011 - ETA: 41s - loss: 15.9376 - acc: 0.011 - ETA: 41s - loss: 15.9378 - acc: 0.011 - ETA: 41s - loss: 15.9381 - acc: 0.011 - ETA: 41s - loss: 15.9384 - acc: 0.011 - ETA: 41s - loss: 15.9387 - acc: 0.011 - ETA: 41s - loss: 15.9390 - acc: 0.011 - ETA: 41s - loss: 15.9367 - acc: 0.011 - ETA: 41s - loss: 15.9370 - acc: 0.011 - ETA: 41s - loss: 15.9373 - acc: 0.011 - ETA: 41s - loss: 15.9376 - acc: 0.011 - ETA: 41s - loss: 15.9379 - acc: 0.011 - ETA: 41s - loss: 15.9381 - acc: 0.0

8688/8688 [==============================] - ETA: 13s - loss: 15.9464 - acc: 0.010 - ETA: 13s - loss: 15.9467 - acc: 0.010 - ETA: 13s - loss: 15.9469 - acc: 0.010 - ETA: 13s - loss: 15.9471 - acc: 0.010 - ETA: 12s - loss: 15.9473 - acc: 0.010 - ETA: 12s - loss: 15.9475 - acc: 0.010 - ETA: 12s - loss: 15.9477 - acc: 0.010 - ETA: 12s - loss: 15.9479 - acc: 0.010 - ETA: 11s - loss: 15.9481 - acc: 0.010 - ETA: 11s - loss: 15.9483 - acc: 0.010 - ETA: 11s - loss: 15.9485 - acc: 0.010 - ETA: 11s - loss: 15.9487 - acc: 0.010 - ETA: 10s - loss: 15.9489 - acc: 0.010 - ETA: 10s - loss: 15.9491 - acc: 0.010 - ETA: 10s - loss: 15.9493 - acc: 0.010 - ETA: 10s - loss: 15.9476 - acc: 0.010 - ETA: 9s - loss: 15.9478 - acc: 0.010 - ETA: 9s - loss: 15.9480 - acc: 0.01 - ETA: 9s - loss: 15.9482 - acc: 0.01 - ETA: 9s - loss: 15.9465 - acc: 0.01 - ETA: 8s - loss: 15.9467 - acc: 0.01 - ETA: 8s - loss: 15.9469 - acc: 0.01 - ETA: 8s - loss: 15.9471 - acc: 0.01 - ETA: 7s - loss: 15.9473 - acc: 0.01 - ETA: 7s - 

In [ ]:
y_train

In [ ]:
model.save('Inception model.h5')

### 

In [ ]:
submission.columns = one_hot.columns

In [ ]:
submission.insert(0, 'id', test_labels['id'])

In [ ]:
submission.to_csv('submission_vgg19.csv', index= False)

In [ ]:
submission.shape